In [1]:
import sqlite3
import requests

# Connect to SQLite database
conn = sqlite3.connect('finance1.db')
cursor = conn.cursor()

# Create tables if they don't exist
cursor.execute('''
CREATE TABLE IF NOT EXISTS Stocks (
    stock_id INTEGER PRIMARY KEY AUTOINCREMENT,
    symbol TEXT UNIQUE NOT NULL,
    company_name TEXT,
    sector TEXT,
    industry TEXT
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS StockPrices (
    price_id INTEGER PRIMARY KEY AUTOINCREMENT,
    stock_id INTEGER,
    date DATE,
    open REAL,
    high REAL,
    low REAL,
    close REAL,
    volume INTEGER,
    FOREIGN KEY (stock_id) REFERENCES Stocks(stock_id),
    UNIQUE (stock_id, date) -- Ensures unique entries for each stock on each date
)
''')

# Function to insert stock data
def insert_stock_data(stocks):
    for stock in stocks:
        cursor.execute('''
            INSERT OR IGNORE INTO Stocks (symbol, company_name, sector, industry)
            VALUES (?, ?, ?, ?)
        ''', (stock['symbol'], stock['company_name'], stock['sector'], stock['industry']))
    conn.commit()

# Function to fetch and insert stock price data from Alpha Vantage
def fetch_and_insert_stock_prices(symbols):
    API_KEY = 'my_key';
    for symbol in symbols:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&apikey={API_KEY}'
        response = requests.get(url)
        data = response.json()
        
        # Extracting the daily time series data
        time_series = data.get('Time Series (Daily)', {})
        
        stock_id = cursor.execute('SELECT stock_id FROM Stocks WHERE symbol=?', (symbol,)).fetchone()[0]
        #fetch the stock_id based on what it is in the StockPrice Table
        
        for date, prices in time_series.items():
            try:
                cursor.execute('''
                    INSERT INTO StockPrices (stock_id, date, open, high, low, close, volume)
                    VALUES (?, ?, ?, ?, ?, ?, ?)
                ''', (stock_id, date, prices['1. open'], prices['2. high'], prices['3. low'], prices['4. close'], prices['5. volume']))
            except sqlite3.IntegrityError:
                # Duplicate entry detected, skipping insertion
                pass
    conn.commit()

# Main function to insert stock and price data
def main():
    stocks = [
        {'symbol': 'NVDA', 'company_name': 'NVIDIA Corporation', 'sector': 'Technology', 'industry': 'Semiconductors'},
        {'symbol': 'TSLA', 'company_name': 'Tesla Inc.', 'sector': 'Consumer Cyclical', 'industry': 'Auto Manufacturers'},
        {'symbol': 'AAPL', 'company_name': 'Apple Inc.', 'sector': 'Technology', 'industry': 'Consumer Electronics'},
        {'symbol': 'MSFT', 'company_name': 'Microsoft Corp.', 'sector': 'Technology', 'industry': 'Software'}
        # Add more stocks here
    ]
    
    symbols = [stock['symbol'] for stock in stocks]
    
    # Insert stock data
    insert_stock_data(stocks)
    
    # Fetch and insert stock prices for all symbols
    fetch_and_insert_stock_prices(symbols)

# Run the main function
if __name__ == '__main__':
    main()

# Close the connection
conn.close()